<a href="https://colab.research.google.com/github/sbehmanesh/DecisionTree_AI/blob/main/ID3_decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ابتدا می بایست داده ها از اکسل خوانده شده و سپس برچسب ها استخراج شوند

In [461]:
from google.colab import drive
import pandas as pd
import math
import numpy as np
import random

tree_depth = 6
learn_ratio = 0.8

drive.mount('/content/drive')
raw_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/prison_dataset.csv')
column_names = raw_data.columns
target_col = column_names[-1]
final_tree = {}
labels = {}
for column_name in column_names:
  labels[column_name] = list(set(raw_data[column_name].values))

print("labels : ",labels)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
labels :  {'Fiscal Year Released': [2010, 2013, 2015], 'Recidivism Reporting Year': [2016, 2018, 2013], 'Race - Ethnicity': ['Black', 'White'], 'Age At Release': ['<45', '>45'], 'Convicting Offense Classification': ['D Felony', 'Other Felony'], 'Convicting Offense Type': ['Drug', 'Violent', 'Other'], 'Convicting Offense Subtype': ['Trafficking', 'Other'], 'Main Supervising District': ['3JD', '5JD'], 'Release Type': ['Discharged End of Sentence', 'Parole'], 'Part of Target Population': ['Yes', 'No'], 'Recidivism - Return to Prison numeric': [0, 1]}


تقسیم داده های به داده های آموزش و تست با نسبت 80 و 20

In [462]:
total_row_number = len(list(raw_data[target_col].values))
learning_data_count = math.floor(learn_ratio*total_row_number)
test_data_count = total_row_number - learning_data_count
learning_data_indexes = random.sample(range(total_row_number), learning_data_count)
test_data_indexes = []

for i in range(total_row_number):
  if i not in learning_data_indexes:
    test_data_indexes.append(i)

print("learn indexes : ",learning_data_indexes)
print("test  indexes : ",test_data_indexes)

learning_data = {}
test_data = {}
for col in column_names:
  learning_data[col] = np.asarray(list(raw_data[col].values))[learning_data_indexes].tolist()
  test_data[col] = np.asarray(list(raw_data[col].values))[test_data_indexes].tolist()

learn indexes :  [5072, 13640, 3210, 8427, 8963, 6496, 255, 5731, 11067, 3489, 8107, 2176, 14156, 704, 13339, 15103, 3835, 3220, 10370, 14618, 11161, 7127, 2604, 1792, 5419, 8173, 14093, 6116, 11656, 5556, 11639, 14272, 3894, 3823, 2225, 7465, 1031, 5199, 11839, 5639, 5464, 13185, 6761, 5458, 11678, 12495, 14530, 11064, 2133, 11167, 8138, 1593, 9208, 14823, 8534, 2466, 12815, 1198, 4540, 11107, 8065, 11436, 15358, 8156, 5966, 4647, 14270, 12566, 514, 6662, 4981, 4016, 15408, 11028, 15420, 13041, 3375, 11024, 10524, 5471, 2626, 2066, 9118, 6589, 11524, 9921, 1102, 2292, 1409, 7148, 5026, 15190, 6273, 3840, 2500, 9726, 8697, 5245, 1356, 7797, 854, 11673, 10774, 14969, 8779, 7333, 8959, 1584, 7150, 14109, 9698, 9409, 6934, 15270, 2184, 839, 4986, 8642, 12237, 8118, 13549, 9711, 13105, 1036, 9957, 559, 5645, 2245, 6134, 11226, 1514, 12833, 10813, 9714, 1303, 12884, 5482, 4188, 3927, 6224, 13936, 8755, 470, 2588, 6403, 2814, 10244, 13387, 13227, 7104, 8996, 14122, 8686, 9939, 7331, 12253, 6

حال باید آنتروپی را محاسبه کنیم

In [463]:
entropy = 0
target_col = column_names[-1]
total_count = len(list(learning_data[target_col]))
for label in labels[target_col]:
  repeat_num = list(learning_data[target_col]).count(label)
  if repeat_num > 0:
    entropy += -(repeat_num/total_count)*math.log2(repeat_num/total_count)

print("total entropy : ",entropy)

total entropy :  0.9881548545805574


حال تابعی می نویسیم که آنتروپی را محاسبه کند

In [464]:
def ent(target_labels,col_label,col_values,target_data):
  ent = 0
  selected_target_leaf = False
  nparray = np.array(col_values)
  target_to_search_row_indexes = np.where(nparray == col_label)[0]
  taget_data_for_label = np.asarray(target_data)[target_to_search_row_indexes].tolist() 
  if len(taget_data_for_label) > 0:
    for target in target_labels:
      ratio = taget_data_for_label.count(target)/len(taget_data_for_label)
      if ratio > 0:
        ent += -ratio*math.log2(ratio)
      else :
        selected_target_leaf = target

  return ent , len(taget_data_for_label) , selected_target_leaf

با استفاده از تابع بالا تابع بهره اطلاعات را بدست می آوریم

In [465]:
def info_gain(parent_entropy,target_labels,col_labels,col_data,target_data):
  gain = parent_entropy
  for label in col_labels:
    entro , data_number , leaf = ent(target_labels,label,list(col_data),list(target_data))
    gain -= (data_number/len(target_data))*entro
  return gain

حال می توانیم بهره اطلاعات را محسابه کنیم

In [466]:
gains = {}
for col in column_names[0:-1]:
  gain = info_gain(entropy,labels[target_col],labels[col],list(learning_data[col]),list(learning_data[target_col]))
  gains[col] = gain

sorted_gains = sorted(gains.items(), key = lambda kv:(kv[1], kv[0]))
root = sorted_gains[-1][0]
print("root => ",root," : gain = ",sorted_gains[-1][1])

root =>  Recidivism Reporting Year  : gain =  0.2138685671719125


حال برای هر زیر شاخه می خواهیم گره بعدی را مشخص کنیم

In [467]:
column_names = list(column_names)
already_used_cols = {root:[0]}
final_tree[root] = {}

def recursive_id3(root,labels,column_names,learning_data,target_col,input_target_to_search_row_indexes,final_tree,depth = 0):
  global already_used_cols
  global tree_depth
  if depth > tree_depth:
    return 0

  branches = labels[root]

  for branch in branches:
    # print("1. branch : ",branch)
    nparray = np.array(list(learning_data[root]))
    target_to_search_row_indexes = np.where(nparray == branch)[0]
    # print("2. indexes : ",target_to_search_row_indexes)
    target_to_search_row_indexes = np.intersect1d(target_to_search_row_indexes,input_target_to_search_row_indexes)
    # print("3. input indexes : ",input_target_to_search_row_indexes)
    # print("4. common indexes : ",target_to_search_row_indexes)
    target_data_for_label = np.asarray(list(learning_data[target_col]))[target_to_search_row_indexes].tolist()
    col_data_for_label    = np.asarray(list(learning_data[root]))[target_to_search_row_indexes].tolist()
    
    if depth == tree_depth:
      # We should now calculate which target label has more popularity
      if len(target_data_for_label): 
        most_popular_target = max(set(target_data_for_label), key = target_data_for_label.count)
        final_tree[root][branch] = most_popular_target
        print("Last depth leaf : ",most_popular_target)
    else:
      entropy = 0
      leaf_type = -1
      total_count = len(list(target_data_for_label))
      # print("total : ",total_count)
      for label in labels[target_col]:
        repeat_num = list(target_data_for_label).count(label)
        if repeat_num > 0:
          entropy  += -(repeat_num/total_count)*math.log2(repeat_num/total_count)
          leaf_type = label

      # print("entropy : ",entropy)
      if entropy == 0:  
        # is leaf
        print(depth,") label -> ",branch," is leaf : ",leaf_type)
        if leaf_type != -1 :
          final_tree[root][branch] = leaf_type
      else:
        gains = {}
        for col in column_names[0:-1]:
          if col in already_used_cols.keys():
            if depth not in already_used_cols[col] and len(already_used_cols[col]) > 0:
              continue
          gain = info_gain(entropy,labels[target_col],labels[col],col_data_for_label,target_data_for_label)
          gains[col] = gain
          # print(gains)

        if len(gains.keys()) > 0:
          sorted_gains = sorted(gains.items(), key = lambda kv:(kv[1], kv[0]),reverse=True)
          # print(sorted_gains)
          print(depth,") label -> ",branch," node : ",sorted_gains[-1][0]," : ",sorted_gains[-1][1])
          if sorted_gains[-1][0] in already_used_cols.keys():
            already_used_cols[sorted_gains[-1][0]].append(depth)
          else:
            already_used_cols[sorted_gains[-1][0]] = [depth]

          final_tree[root][branch] = {sorted_gains[-1][0]:{}}
          recursive_id3(sorted_gains[-1][0],labels,column_names,learning_data,target_col,target_to_search_row_indexes.copy(),final_tree[root][branch],depth + 1)
        else: 
          print(depth,") Tree ended here")


target_to_search_row_indexes = list(range(0,len(list(learning_data[root]))))
recursive_id3(root,labels,column_names.copy(),learning_data,target_col,target_to_search_row_indexes.copy(),final_tree,1)

1 ) label ->  2016  node :  Age At Release  :  0.8189735733282562
2 ) label ->  <45  node :  Convicting Offense Classification  :  0.7816709834196973
3 ) label ->  D Felony  node :  Convicting Offense Subtype  :  0.8265758146026101
4 ) label ->  Trafficking  node :  Convicting Offense Type  :  0.7932698344258596
5 ) label ->  Drug  node :  Fiscal Year Released  :  0.7932698344258596


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """


Last depth leaf :  1
5 ) label ->  Violent  is leaf :  -1
5 ) label ->  Other  is leaf :  -1
4 ) label ->  Other  node :  Convicting Offense Type  :  0.0
5 ) label ->  Drug  node :  Fiscal Year Released  :  0.6684725699243226
Last depth leaf :  1
5 ) label ->  Violent  node :  Fiscal Year Released  :  0.9372344385127174
Last depth leaf :  1
5 ) label ->  Other  node :  Fiscal Year Released  :  0.8130607875117535
Last depth leaf :  1
3 ) label ->  Other Felony  node :  Convicting Offense Subtype  :  0.7491856012504546
4 ) label ->  Trafficking  node :  Convicting Offense Type  :  0.7549694116467962
5 ) label ->  Drug  node :  Fiscal Year Released  :  0.7549694116467962
Last depth leaf :  1
5 ) label ->  Violent  is leaf :  -1
5 ) label ->  Other  is leaf :  -1
4 ) label ->  Other  node :  Convicting Offense Type  :  0.0
5 ) label ->  Drug  node :  Fiscal Year Released  :  0.6136190195993708
Last depth leaf :  1
5 ) label ->  Violent  node :  Fiscal Year Released  :  0.8240815543156681
L

نمایش مناسب درخت تصمیم

In [468]:
def print_nested_dict(dict_obj, indent = 0):
    # Pretty Print nested dictionary with given indent level  
    for key, value in dict_obj.items():
        if isinstance(value, dict):
            print(' ' * indent, key, ':', '{')
            print_nested_dict(value, indent + 4)
            print(' ' * indent, '}')
        else:
            print(' ' * indent, key, ':', value)

print_nested_dict(final_tree)

 Recidivism Reporting Year : {
     2016 : {
         Age At Release : {
             <45 : {
                 Convicting Offense Classification : {
                     D Felony : {
                         Convicting Offense Subtype : {
                             Trafficking : {
                                 Convicting Offense Type : {
                                     Drug : {
                                         Fiscal Year Released : {
                                             2013 : 1
                                         }
                                     }
                                 }
                             }
                             Other : {
                                 Convicting Offense Type : {
                                     Drug : {
                                         Fiscal Year Released : {
                                             2013 : 1
                                         }
                                

حال به اجرا درخت تصمیم ایجاد شده روی داده های تست می پردازیم

In [469]:
def recursive_validate_result_by_tree(tree,test_data,test_data_index,raw_test_data):
  if tree not in [0,1]:
    tree_keys = list(tree.keys())
    if len(tree_keys) > 1:
      # branches level
      for key in tree_keys:
        if key == test_data[test_data_index]:
          # print("leaf : ",key)
          return recursive_validate_result_by_tree(tree[key],[],test_data_index,raw_test_data)
    elif len(tree_keys) == 1:
      # node level
      # print("node : ",tree[tree_keys[0]])
      if tree_keys[0] in raw_test_data.keys():
        return recursive_validate_result_by_tree(tree[tree_keys[0]],raw_test_data[tree_keys[0]],test_data_index,raw_test_data)
      return "not trained"
  else:
    # last depth of tree
    # print(tree)
    return tree

pt = 0
pf = 0
nt = 0
nf = 0
not_trained = 0
for i in range(test_data_count):
  result = recursive_validate_result_by_tree(final_tree,[],i,test_data)
  if result != "not trained":
    if result == test_data[target_col][i]:
      if result > 0:
        pt += 1
      else:
        nt += 1
    elif result != test_data[target_col][i]:
      if result > 0:
        pf += 1
      else:
        nf += 1
  else:
    not_trained += 1

  # print(i,") predicted : ",result," - real : ",test_data[target_col][i])

محاسبه ماتریس آشفتگی و دقت درخت

In [470]:
print("Confusion Matrix")
print("                 real true   real false")
print("                ------------------------")
print("predicted true | ",pt,"        ",pf)
print("predicted false| ",nf,"        ",nt)
print("                ------------------------")
accuracy = ((pt + nt)/test_data_count)*100
print("accuracy : ",accuracy,"%")
print("data could not be validated : ",not_trained)

Confusion Matrix
                 real true   real false
                ------------------------
predicted true |  473          0
predicted false|  0          0
                ------------------------
accuracy :  15.332252836304699 %
data could not be validated :  2612
